# Assignment 8: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A8-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [ ]:
!pip install featuretools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__



'1.23.0'

In [ ]:
# list all rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")

# could drop columns here, if needed

print(users["label"].value_counts())
print(orders.shape, order_products.shape)
users.head()

False    628
True     139
Name: label, dtype: int64
(5997, 4) (57780, 8)


,Unnamed: 0,user_id,label
0,0,1,False
1,1,2,True
2,2,3,False
3,3,7,False
4,4,10,False


## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [ ]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    # --- Write your code below ---

    df = {'users': (users,'user_id'),
                  'orders': (orders, 'order_id'),
                  'order_products':(order_products,'order_product_id')}

    relations = [('orders','order_id','order_products','order_id'),('users','user_id','orders','user_id')]
    # return the EntitySet object
    return ft.EntitySet('entity-set', df, relations)

### 1.2 Deep Feature Synthesis

In [ ]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

/usr/local/lib/python3.9/dist-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(


In [ ]:
# use ft.dfs to perform feature engineering
# --- Write your code below ---

features, feature_names = ft.dfs(entityset = es, target_dataframe_name = 'users')

In [ ]:
# output what features you generate
features

,Unnamed: 0,label,COUNT(orders),MAX(orders.Unnamed: 0),MEAN(orders.Unnamed: 0),MIN(orders.Unnamed: 0),SKEW(orders.Unnamed: 0),STD(orders.Unnamed: 0),SUM(orders.Unnamed: 0),COUNT(order_products),MAX(order_products.Unnamed: 0),MAX(order_products.aisle_id),MAX(order_products.reordered),MEAN(order_products.Unnamed: 0),MEAN(order_products.aisle_id),MEAN(order_products.reordered),MIN(order_products.Unnamed: 0),MIN(order_products.aisle_id),MIN(order_products.reordered),MODE(order_products.department),MODE(order_products.product_name),NUM_UNIQUE(order_products.department),NUM_UNIQUE(order_products.product_name),SKEW(order_products.Unnamed: 0),SKEW(order_products.aisle_id),SKEW(order_products.reordered),STD(order_products.Unnamed: 0),STD(order_products.aisle_id),STD(order_products.reordered),SUM(order_products.Unnamed: 0),SUM(order_products.aisle_id),SUM(order_products.reordered),MAX(orders.COUNT(order_products)),MAX(orders.MEAN(order_products.Unnamed: 0)),MAX(orders.MEAN(order_products.aisle_id)),MAX(orders.MEAN(order_products.reordered)),MAX(orders.MIN(order_products.Unnamed: 0)),MAX(orders.MIN(order_products.aisle_id)),MAX(orders.MIN(order_products.reordered)),MAX(orders.NUM_UNIQUE(order_products.department)),MAX(orders.NUM_UNIQUE(order_products.product_name)),MAX(orders.SKEW(order_products.Unnamed: 0)),MAX(orders.SKEW(order_products.aisle_id)),MAX(orders.SKEW(order_products.reordered)),MAX(orders.STD(order_products.Unnamed: 0)),MAX(orders.STD(order_products.aisle_id)),MAX(orders.STD(order_products.reordered)),MAX(orders.SUM(order_products.Unnamed: 0)),MAX(orders.SUM(order_products.aisle_id)),MAX(orders.SUM(order_products.reordered)),MEAN(orders.COUNT(order_products)),MEAN(orders.MAX(order_products.Unnamed: 0)),MEAN(orders.MAX(order_products.aisle_id)),MEAN(orders.MAX(order_products.reordered)),MEAN(orders.MEAN(order_products.Unnamed: 0)),MEAN(orders.MEAN(order_products.aisle_id)),MEAN(orders.MEAN(order_products.reordered)),MEAN(orders.MIN(order_products.Unnamed: 0)),MEAN(orders.MIN(order_products.aisle_id)),MEAN(orders.MIN(order_products.reordered)),MEAN(orders.NUM_UNIQUE(order_products.department)),MEAN(orders.NUM_UNIQUE(order_products.product_name)),MEAN(orders.SKEW(order_products.Unnamed: 0)),MEAN(orders.SKEW(order_products.aisle_id)),MEAN(orders.SKEW(order_products.reordered)),MEAN(orders.STD(order_products.Unnamed: 0)),MEAN(orders.STD(order_products.aisle_id)),MEAN(orders.STD(order_products.reordered)),MEAN(orders.SUM(order_products.Unnamed: 0)),MEAN(orders.SUM(order_products.aisle_id)),MEAN(orders.SUM(order_products.reordered)),MIN(orders.COUNT(order_products)),MIN(orders.MAX(order_products.Unnamed: 0)),MIN(orders.MAX(order_products.aisle_id)),MIN(orders.MAX(order_products.reordered)),MIN(orders.MEAN(order_products.Unnamed: 0)),MIN(orders.MEAN(order_products.aisle_id)),MIN(orders.MEAN(order_products.reordered)),MIN(orders.NUM_UNIQUE(order_products.department)),MIN(orders.NUM_UNIQUE(order_products.product_name)),MIN(orders.SKEW(order_products.Unnamed: 0)),MIN(orders.SKEW(order_products.aisle_id)),MIN(orders.SKEW(order_products.reordered)),MIN(orders.STD(order_products.Unnamed: 0)),MIN(orders.STD(order_products.aisle_id)),MIN(orders.STD(order_products.reordered)),MIN(orders.SUM(order_products.Unnamed: 0)),MIN(orders.SUM(order_products.aisle_id)),MIN(orders.SUM(order_products.reordered)),MODE(orders.DAY(order_time)),MODE(orders.MODE(order_products.department)),MODE(orders.MODE(order_products.product_name)),MODE(orders.MONTH(order_time)),MODE(orders.WEEKDAY(order_time)),MODE(orders.YEAR(order_time)),NUM_UNIQUE(orders.DAY(order_time)),NUM_UNIQUE(orders.MODE(order_products.department)),NUM_UNIQUE(orders.MODE(order_products.product_name)),NUM_UNIQUE(orders.MONTH(order_time)),NUM_UNIQUE(orders.WEEKDAY(order_time)),NUM_UNIQUE(orders.YEAR(order_time)),SKEW(orders.COUNT(order_products)),SKEW(orders.MAX(order_products.Unnamed: 0)),SKEW(orders.MAX(order_products.aisle_id)),SKEW(orders.MAX(order_products.reordered)),SKEW(orders.MEAN(order_

## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [ ]:
# --- Write your code below ---
# Select top-10 features and return X, y (X.shape = (767, 10)
y = features.reset_index()['label']
#display(y)
df_new = features.reset_index()
df_cor = df_new.corr(method='spearman')
#display(df_cor)
df_cor
df_cor_label = pd.DataFrame(df_cor['label'].dropna())
df_cor_label = df_cor_label.reset_index()
#display(df_cor_label)
df_cor_label.rename(columns = {'label':'score'},inplace = True)
#display(df_cor_label)
feature_names = df_cor_label.sort_values(by= 'score', ascending = False)['index'][1:11]
feature_names = feature_names.tolist()

X= df_new[feature_names]
#display(X)
X.shape

(767, 10)

### 2.2 Get accuracy and list features

In [ ]:
X = X.dropna()
y = y.loc[X.index]

clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf, X=X, y=y, cv=3, scoring="roc_auc", verbose=True, error_score='raise')
print("AUC %.2f" % (scores.mean()))

# Print top-10 features

for features in feature_names:
  print(features)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.65
MEAN(orders.COUNT(order_products))
MEAN(orders.NUM_UNIQUE(order_products.product_name))
MEAN(orders.SUM(order_products.aisle_id))
SUM(orders.NUM_UNIQUE(order_products.product_name))
COUNT(order_products)
MEAN(orders.STD(order_products.Unnamed: 0))
SUM(order_products.aisle_id)
NUM_UNIQUE(order_products.product_name)
SUM(orders.STD(order_products.Unnamed: 0))
MIN(orders.COUNT(order_products))


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.7s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

--- Write your answer here---
<br>**Advantages**
1. Features explorations can be much faster than manually generation.
2. Features explorations are suitable for processing large datasets.
3. Only a few entities can generate large amount of features with different transformations.

<br>**Disadvantages**
1. Invalid features can be generated before feeding into models.
2. For small datasets, features generation can be a little complex.
3. If a large number of features are generated, it can result in the model being poorly trained unless there is an abundance of data available.

<br>**Solutions to improve**
1. To avoid generating invalid features, we might examine and preprocess the data carefully before utilizing featuretools. 
2. We can experiment with different aggregation and transformation functions such as feature scaling as feature engineering techniques.
3. Using techniques such as correlation analysis and regularization methods to identify the most informative features.

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 8.